In [27]:
sentences = ["I like dog", "I love coffee", "I hate milk", "You like cat", "You love milk", "You hate coffee"]
import numpy 

class RNN :
    #RNN(x) = h*Wh + x*Wx + b
    
    def __init__(self, input_size, output_size) :    
        self.Wh = numpy.random.randn(output_size, output_size)
        self.Wx = numpy.random.randn(input_size, output_size)
        self.b  = numpy.random.randn(1, output_size)
        self.hs = None
        self.cache = None #x 저장공간 #(1,in)

    def forward(self, x) :
        self.cache =  x
        self.hs = []
        for b in range(x.shape[0]) :
            temp = []
            h = numpy.zeros(shape=(1,self.Wx.shape[1]))
            for s in range(x.shape[1]) :
                h = numpy.tanh(numpy.dot(h, self.Wh) + numpy.dot(x[b,s:s+1,:], self.Wx) + self.b) #(1,out)
                temp.append(h[0])
            self.hs.append(temp)
        self.hs = numpy.array(self.hs)
        return self.hs;

    def backward(self, dhs, lr) :     
        dWh = numpy.zeros_like(self.Wh) #(out, out)
        dWx = numpy.zeros_like(self.Wx) #(in, out)
        db  = numpy.zeros_like(self.b)  #(1, out) 
        dxs = []        
        for b in range(self.cache.shape[0]) :   
            temp = []
            dh = dhs = dhs[b:b+1,:]
            for s in range(self.cache.shape[1]-1,-1,-1) :
                dt = dh * (1 - (self.hs[b,s:s+1,:] ** 2)) #tanh 미분 # = (1, out)
                if  s != 0 :
                    dWh = dWh + numpy.dot(self.hs[b,s-1:s,:].T, dt)
                # else :
                #     dWh = dWh + numpy.dot(영행렬, dt) #0을 왜더함?
                dWx = dWx + numpy.dot(self.cache[b,s:s+1,:].T, dt)  #(1,out) @ (1,in) = (in,out) 
                db  = db  + dt # = (1,out)
                dh  = numpy.dot(dt, self.Wh.T) #(1,out) @ (out, out) = (1, out)
                dx  = numpy.dot(dt, self.Wx.T) #(1, out) @ (in, out) = (1, in)
                temp.append(dx[0])
            dxs.append(temp)
        self.Wh = self.Wh - dWh * lr / self.cache.shape[0]
        self.Wx = self.Wx - dWx * lr / self.cache.shape[0]
        self.b  = self.b  - db  * lr / self.cache.shape[0]
        dxs = numpy.array(dxs)
        return dxs;

class Linear :
    #Linear(x) = x*W + b
    def __init__(self, input_size, output_size) :
        self.W = numpy.random.randn(input_size, output_size) #(in, out)
        self.b = numpy.random.randn(1, output_size) #(n, out)
        self.cache = None #x 저장공간 #(n,in)

    def forward(self, x) :
        self.cache = x
        y = numpy.dot(x, self.W) + self.b #(n,out)
        return y

    def backward(self, dy, lr) :
        dW = numpy.dot(self.cache.T, dy) #(n,out) @ (n,in) = (in, out)
        db = dy
        dx = numpy.dot(dy, self.W.T) #(n, out) @ (in, out) = (n, in)
        
        self.W = self.W - dW * lr
        self.b = self.b - db * lr
        return dx;

class Softmax :
    def __init__(self) :
        pass

    def forward(self, x) :
        exp = numpy.exp(x-numpy.max(x))
        return exp / numpy.sum(exp)

    def backward(self, dy, lr = 0) :
        return dy
    
class CEE :
    def __init__(self) :
        self.cache_y = None #y값 저장
        self.cache_t = None #t값 저장
        
    def forward(self, y, t) :
        eps = 1e-15
        self.cache_y = y
        self.cache_t = t
        return -1 * numpy.sum(t * numpy.log(y + eps))
        
    def backward(self) :
        dy = self.cache_y - self.cache_t
        return dy
    

In [31]:
### 다음에 올 단어 맞추는 AI

##데이터 전처리
#1) 원 핫 인코딩
data = list(set(" ".join(sentences).split()))
#print(data)
word_dict = {w : i for i,w in enumerate(data)}
number_dict = {i : w for i,w in enumerate(data)}

corpus = []
for s in sentences :
    words = s.split()
    s_array = []
    for w in words :
        s_array.append(numpy.eye(len(word_dict))[word_dict[w]])
    corpus.append(s_array)
corpus = numpy.array(corpus)
#print(corpus)

##AI 학습
#1) x, t 입력
x = corpus[:,0:2,:]
t = corpus[:,2,:]

#2) 함수(F(x), 손실함수, 최적화 함수)들 선언
#F(x) :  x -> rnn(x) -> f(x) -> softmax(x) -> y
rnn = RNN(x.shape[2],5)
f = Linear(5, 9)
softmax = Softmax()
loss_function = CEE()
epoch = 200
batch_size = x.shape[0]

for e in range(epoch) :
    loss_sum = 0
    for b in range(batch_size) :
        #3) y = F(x)
        y = rnn.forward(x[b:b+1,:,:])  
        y = y[:,-1,:]
        y = f.forward(y)
        y = softmax.forward(y)
        
        #4) y와 t 비교
        loss = loss_function.forward(y, t[b:b+1])
        loss_sum += loss
        
        #5) F(x) 수정 (=미분)
        lr = 0.3
        dy = loss_function.backward()
        dy = softmax.backward(dy, lr = lr)
        dy = f.backward(dy, lr = lr)
        dy = rnn.backward(dy, lr = lr)

    if (e+1) % 10 == 0 :
        print("epoch {} | loss {}".format(e+1, loss_sum))


epoch 10 | loss 6.729875626276301
epoch 20 | loss 3.604689259448669
epoch 30 | loss 0.5215626533835382
epoch 40 | loss 0.23849208890044288
epoch 50 | loss 0.1561725897645702
epoch 60 | loss 0.11644276248410515
epoch 70 | loss 0.09295692477554303
epoch 80 | loss 0.0774171503016238
epoch 90 | loss 0.06636347823884978
epoch 100 | loss 0.05809309041704652
epoch 110 | loss 0.051669439879303336
epoch 120 | loss 0.0465344018887328
epoch 130 | loss 0.04233450048223021
epoch 140 | loss 0.03883490007751225
epoch 150 | loss 0.03587343794895527
epoch 160 | loss 0.03333450541109151
epoch 170 | loss 0.031133445236398855
epoch 180 | loss 0.02920683382598692
epoch 190 | loss 0.02750621065650516
epoch 200 | loss 0.02599390730935365


In [30]:
#검증
#sentences = ["I like dog", "I love coffee", "I hate milk", "You like cat", "You love milk", "You hate coffee"]

y = rnn.forward(x)  
y = y[:,-1,:]
y = f.forward(y)


for i in y :
    print(number_dict[numpy.argmax(i)])

dog
coffee
milk
cat
milk
coffee
